## Paso 1: Importar las dependencias del proyecto

In [1]:
import time
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.13.0'

## Paso 2: Pre procesado del dataset

### Cargar el dataset del MNIST

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

### Normalización de las imágenes

In [4]:
X_train = X_train / 255.
X_test = X_test / 255.

In [5]:
X_train.shape

(60000, 28, 28)

### Redimensionar el dataset

In [6]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

## Paso 3: Entrenamiento distribuido

### Definir un modelo normal (no distribuido)

In [8]:
model_normal = tf.keras.models.Sequential()

In [9]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))

In [10]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [11]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [12]:
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Definir una estrategia distribuida

In [13]:
distribute = tf.distribute.MirroredStrategy()#Modelo a usar

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


### Definir un modelo distribuido

In [14]:
with distribute.scope():#Definir modelo
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Comparar la velocidad de entrenamiento normal vs distribuida

In [15]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=20, batch_size=20)
print("El entrenamiento distribuido ha tardado: {}".format(time.time() - start_time))

Epoch 1/20
3000/3000 [==============================] - 8s 3ms/step - loss: 0.2733 - sparse_categorical_accuracy: 0.9190
Epoch 2/20
3000/3000 [==============================] - 8s 3ms/step - loss: 0.1350 - sparse_categorical_accuracy: 0.9588
Epoch 3/20
3000/3000 [==============================] - 8s 3ms/step - loss: 0.1042 - sparse_categorical_accuracy: 0.9681
Epoch 4/20
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0846 - sparse_categorical_accuracy: 0.9736
Epoch 5/20
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0724 - sparse_categorical_accuracy: 0.9776
Epoch 6/20
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0633 - sparse_categorical_accuracy: 0.9792
Epoch 7/20
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0571 - sparse_categorical_accuracy: 0.9814
Epoch 8/20
3000/3000 [==============================] - 7s 2ms/step - loss: 0.0535 - sparse_categorical_accuracy: 0.9824
Epoch 9/20
3000/3000 [==========

In [16]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=20, batch_size=20)
print("El entrenamiento normal ha tardado: {}".format(time.time() - start_time))

Epoch 1/20
3000/3000 [==============================] - 7s 2ms/step - loss: 0.2714 - sparse_categorical_accuracy: 0.9202
Epoch 2/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.1357 - sparse_categorical_accuracy: 0.9587
Epoch 3/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.1017 - sparse_categorical_accuracy: 0.9687
Epoch 4/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0838 - sparse_categorical_accuracy: 0.9738
Epoch 5/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0745 - sparse_categorical_accuracy: 0.9763
Epoch 6/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0643 - sparse_categorical_accuracy: 0.9792
Epoch 7/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0574 - sparse_categorical_accuracy: 0.9808
Epoch 8/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0516 - sparse_categorical_accuracy: 0.9829
Epoch 9/20
3000/3000 [==========

In [17]:
test_loss, test_accuracy = model_distributed.evaluate(X_test, y_test)
print("Precisión del conjunto de test Distribuido: {}".format(test_accuracy))
test_loss, test_accuracy = model_normal.evaluate(X_test, y_test)
print("Precisión del conjunto de test Normal: {}".format(test_accuracy))

313/313 [==============================] - 1s 2ms/step - loss: 0.0976 - sparse_categorical_accuracy: 0.9794
Precisión del conjunto de test Distribuido: 0.9793999791145325
313/313 [==============================] - 1s 2ms/step - loss: 0.0958 - sparse_categorical_accuracy: 0.9797
Precisión del conjunto de test Normal: 0.9797000288963318
